In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile
zipfile1='/content/drive/MyDrive/archive.zip'
unzipdirectory='/content/'
with zipfile.ZipFile(zipfile1,'r') as zip_ref:
   zip_ref.extractall(unzipdirectory)

In [3]:
from keras.models import Sequential
from keras.layers import Convolution2D,MaxPooling2D,Dense,Dropout,Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [4]:
# Initialize image data generator with rescaling

validation_data_gen = ImageDataGenerator(rescale=1./255)


In [5]:
from keras.preprocessing.image import ImageDataGenerator

# Define the data augmentation parameters
Train_datagen = ImageDataGenerator(
    rotation_range=10,      # Degree range for random rotations
    width_shift_range=0.1,  # Fraction of total width for random horizontal shift
    height_shift_range=0.1, # Fraction of total height for random vertical shift
    shear_range=0.1,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.2,         # Range for random zoom (e.g., 0.2 => 0.8 to 1.2)
    horizontal_flip=True,   # Randomly flip images horizontally
    rescale=1./255          # Rescale pixel values to [0, 1]
)

In [6]:
# Preprocess all train images
train_generator = Train_datagen.flow_from_directory(
        '/content/train',
        target_size=(48, 48),
        batch_size=32,
        color_mode="grayscale",
        class_mode='categorical'
        )

# Preprocess all test images
validation_generator = validation_data_gen.flow_from_directory(
        '/content/test',
        target_size=(48, 48),
        batch_size=32,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [8]:
import cv2
from keras.layers.convolutional.conv2d_transpose import Conv2D
#creating model structure


EmotionModel = Sequential()

EmotionModel.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
EmotionModel.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
EmotionModel.add(MaxPooling2D(pool_size=(2, 2)))
#EmotionModel.add(Dropout(0.25))

EmotionModel.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
EmotionModel.add(MaxPooling2D(pool_size=(2, 2)))
EmotionModel.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
EmotionModel.add(MaxPooling2D(pool_size=(2, 2)))
#EmotionModel.add(Dropout(0.25))

EmotionModel.add(Flatten())


EmotionModel.add(Dense(1024, activation='relu'))
EmotionModel.add(Dropout(0.5))
EmotionModel.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)
EmotionModel.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001, decay=1e-6), metrics=['accuracy'])

In [11]:
emotion_model_info = EmotionModel.fit(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=30,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

Epoch 1/30
448/448 [==============================] - 307s 686ms/step - loss: 1.5162 - accuracy: 0.4155 - val_loss: 1.3929 - val_accuracy: 0.4731
Epoch 2/30
448/448 [==============================] - 302s 672ms/step - loss: 1.4815 - accuracy: 0.4330 - val_loss: 1.3605 - val_accuracy: 0.4831
Epoch 3/30
448/448 [==============================] - 291s 650ms/step - loss: 1.4478 - accuracy: 0.4392 - val_loss: 1.3427 - val_accuracy: 0.4873
Epoch 4/30
448/448 [==============================] - 298s 666ms/step - loss: 1.4290 - accuracy: 0.4577 - val_loss: 1.3119 - val_accuracy: 0.5036
Epoch 5/30
448/448 [==============================] - 290s 647ms/step - loss: 1.4032 - accuracy: 0.4683 - val_loss: 1.3052 - val_accuracy: 0.5091
Epoch 6/30
448/448 [==============================] - 296s 660ms/step - loss: 1.3898 - accuracy: 0.4708 - val_loss: 1.2684 - val_accuracy: 0.5123
Epoch 7/30
448/448 [==============================] - 295s 658ms/step - loss: 1.3564 - accuracy: 0.4869 - val_loss: 1.2525 -

In [11]:
test_loss, test_acc =EmotionModel.evaluate(validation_generator, verbose=2)

print('\nTest accuracy:', test_acc)

113/113 - 3s - loss: 1.0709 - accuracy: 0.5975 - 3s/epoch - 25ms/step

Test accuracy: 0.5975201725959778


113/113 [==============================] - 3s 29ms/step
